In [4]:
import GalaxyAper

In [5]:
from astropy.io import fits

hdul = fits.open('F160W_cutout.fits')
data = hdul[0].data
header= hdul[0].header
filter_name='F160W'


GalaxyAper.GalaxyAper(data, header, filter_name=filter_name, verbose=True)

Processed galaxy number 0
Processed galaxy number 1
Processed galaxy number 2
Processed galaxy number 3
Processed galaxy number 4
Processed galaxy number 5
Processed galaxy number 6
Processed galaxy number 7
Processed galaxy number 8
Processed galaxy number 9
Processed galaxy number 10
Processed galaxy number 11
Processed galaxy number 12
Processed galaxy number 13
Processed galaxy number 14
Processed galaxy number 15
Processed galaxy number 16
Processed galaxy number 17
Processed galaxy number 18
Processed galaxy number 19
Processed galaxy number 20
Processed galaxy number 21
Processed galaxy number 22
Processed galaxy number 23
Processed galaxy number 24
Processed galaxy number 25
Processed galaxy number 26
Processed galaxy number 27
Processed galaxy number 28
Processed galaxy number 29
Processed galaxy number 30
Processed galaxy number 31
Processed galaxy number 32
Processed galaxy number 33
Processed galaxy number 34
Processed galaxy number 35
Processed galaxy number 36
Processed g

In [6]:
catalog_file='xmm113_catalog_v7.fits'
#Redshift limits for membership
z_low = 1.53
z_high = 1.69
filter_name='F160W'
#You can specify z_choice as either zspec, zphot, or zboth depending on what you want to use for membership determination


GalaxyAper.catalog_matching(catalog_file, z_low=z_low, z_high=z_high, z_choice='zboth')

Using GalaxyAper file: F160W_full_array_iterative.csv
Output written to z_member_catalogue.cat


In [8]:
# Load FITS files into memory
filter_data_map = {
    'F625W': fits.open('F625W_cutout.fits')[0].data,
    'F475W': fits.open('F475W_cutout.fits')[0].data,
    # Add more filters as needed
}

# Call the function
GalaxyAper.GalaxyAper_multi_filter(data, filter_data_map)

Using catalog_matching file: z_member_catalogue.cat
Processing filter: F625W
Processing filter: F475W
